In [1]:
import sys
import numpy as np
import pandas as pd

print(sys.version)

ModuleNotFoundError: No module named 'pandas'

# Markov decision Process

## Markov process
### 시스템 상태 (state)와 변환 행렬 (transition_matrix)

마코프 프로세스 또는 마코프 체인인 어떤 시스템에서는, 유한한 개수의 상태들이 일정한 확률로 다른 상태로 변환된다. 또한, 마코프 속성(Markov property)을 충족하기 위해서는, 시스템의 미래 상태는 현재 상태에만 영향을 받아야 한다. 이러한 속성은 마코스 시스템의 각 상태가 다른 것들과 서로 구분되는 고유함을 보장하기 위해서이다.


In [11]:
state_space = [0, 1]
transition_matrix = [[0.8, 0.2], [0.1, 0.9]]
pd.DataFrame(transition_matrix, index=state_space, columns=state_space)

,0,1
0,0.8,0.2
1,0.1,0.9


### 히스토리(history)와 에피소드(episode)

상태에 대한 일련의 변환 기록을 히스토리라고 한다. 히스토리 내 임의의 일련의 구간을 에피소드라고 한다. 각 에피소드는 변환 행렬의 확률에 따라 일련의 상태 변환으로 구성된다. 

In [12]:
import numpy.random as rnd

def generate_episode(state_space, transition_matrix, n_steps, initial_state=None):
    episode = [rnd.choice(state_space) if initial_state is None else initial_state]
    for _ in range(n_steps):
        next_state = rnd.choice(state_space, p=transition_matrix[episode[-1]])
        episode.append(next_state)
    return episode

episode = generate_episode(state_space, transition_matrix, 1000)
episode[:10]

[0, 1, 1, 1, 1, 1, 1, 1, 1, 1]

#### Episodes

시스템 상태에 대한 관찰을 에피소드(episode)라고 한다.

In [13]:
for _ in range(10):
    episode = generate_episode(state_space, transition_matrix, 10)
    print(episode)


[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1]
[0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1]
[0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1]
[0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1]
[0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1]
[1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1]


In [14]:
episodes = list(zip(episode, episode[1:]))
estimated_probabilities = pd.Series(episodes).value_counts() / len(episodes)
for i, j in estimated_probabilities.index:
    print(f'{i} -> {j}: {estimated_probabilities[i, j]}')

pd.DataFrame([[estimated_probabilities[i, j] for j in state_space] for i in state_space])

1 -> 1: 0.5
0 -> 0: 0.3
1 -> 0: 0.1
0 -> 1: 0.1


,0,1
0,0.3,0.1
1,0.1,0.5


## Markov reward process

$$ G_t = R_{t+1} + \gamma R_{t+2} + \gamma^2 R_{t+3} + \cdots = \sum_{k=0}^{\infty}\gamma^kR_{t+k+1} $$

In [15]:
state_space = [0, 1, 2, 3]
space_names = ['Home', 'Coffee', 'Chat', 'Work']
transition_matrix = [[0.6, 0.4, 0.0, 0.0], [0.0, 0.1, 0.7, 0.2], [0.0, 0.2, 0.5, 0.3], [0.2, 0.2, 0.1, 0.5]]
pd.DataFrame(transition_matrix, index=space_names, columns=space_names)

,Home,Coffee,Chat,Work
Home,0.6,0.4,0.0,0.0
Coffee,0.0,0.1,0.7,0.2
Chat,0.0,0.2,0.5,0.3
Work,0.2,0.2,0.1,0.5


In [16]:
rewards = {(0, 0): 1.0, (0, 1): 1.0, (3, 3): 5.0, (3, 2): -3.0, (2, 3): 2.0, (3, 1): 1.0, (1, 3): 3.0, (1, 1): 1.0, (1, 2): 2.0, (2, 1): 1.0, (2, 2): -1.0, (3, 0): 2}
pd.Series(rewards).sort_index()

0  0    1.0
   1    1.0
1  1    1.0
   2    2.0
   3    3.0
2  1    1.0
   2   -1.0
   3    2.0
3  0    2.0
   1    1.0
   2   -3.0
   3    5.0
dtype: float64

In [17]:
def compute_return(episode, reward, discount_factor=0.0):
    return_ = 0.0
    for i in range(len(episode) - 1):
        current_state, next_state = episode[i], episode[i + 1]
        return_ += discount_factor ** i * reward.get((current_state, next_state), 0.0)
    return return_

episode = generate_episode(state_space, transition_matrix, 20)
print(f'For episode: {episode}')
for gamma in np.arange(0.0, 1.1, 0.1):
    print(f'Return (gamma={gamma:.1f}): {compute_return(episode, rewards, discount_factor=gamma):.2f}')

For episode: [3, 3, 3, 0, 0, 1, 2, 3, 0, 0, 0, 1, 2, 2, 2, 3, 3, 3, 0, 0, 0]
Return (gamma=0.0): 5.00
Return (gamma=0.1): 5.52
Return (gamma=0.2): 6.09
Return (gamma=0.3): 6.72
Return (gamma=0.4): 7.44
Return (gamma=0.5): 8.30
Return (gamma=0.6): 9.41
Return (gamma=0.7): 11.00
Return (gamma=0.8): 13.74
Return (gamma=0.9): 20.00
Return (gamma=1.0): 39.00


### Value of the state

$$ V(s) = \mathbb{E}[\,G|S_t=s]\, $$

In [18]:
state_value = lambda state: np.sum(np.array(transition_matrix[state]) * np.array([compute_return((state, i), rewards, 0) for i in state_space]))
for i in state_space:
    print(f'State value for {i}: {state_value(i):.2f}')

State value for 0: 1.00
State value for 1: 2.10
State value for 2: 0.30
State value for 3: 2.80
